In [ ]:
# python3 -m pip install torch==1.8.2+cu102 torchvision==0.9.2+cu102 torchaudio==0.8.2 -f https://download.pytorch.org/whl/lts/1.8/torch_lts.html
# python3 -m pip install simpletransformers
# python3 -m pip install pandas scikit-learn

In [1]:
import pandas as pd
from sklearn.preprocessing import LabelEncoder
import torch
from transformers import AutoTokenizer, AutoModelForSequenceClassification
from sklearn.metrics import confusion_matrix, classification_report,f1_score
import numpy as np
from scipy.spatial.distance import cdist
import heapq
from collections import Counter
EVALMODE=1

In [2]:
df_train = pd.read_csv("../data/classification/train/train_tamil_offensive_binary.tsv",sep="\t")
df_train= df_train.rename(columns={'0': 'text',"1":"labels"})
df_train

,text,labels
0,movie vara level la Erika poguthu,0
1,Padam nalla comedy padama irukum polaye..,0
2,karthick subburaj anne .... intha padam vetri ...,0
3,கவுண்டர் தேவர்.சார்பாக வெற்றி பெற வாழ்த்துக்கள் 🦁,0
4,ippo intha trailer ah parkuravana oru like pod...,0
...,...,...
33680,inda pathi like thala fans um thandrukanga da ...,1
33681,Trending number #2 idhukku nammalam karanamnu ...,0
33682,"Movie script super, athuvum HIP HOP Tamizha mu...",0
33683,Just 3k likes for 300k likes,0


In [3]:
df_test = pd.read_csv("../data/classification/test/test_tamil_offensive_binary.tsv",sep="\t")
df_test

,text,labels
0,14.12.2018 epo trailer pathutu irken ... Semay...,0
1,Paka thana poro movie la Enna irukunu,0
2,Suriya anna vera level anna mass,0
3,suma kaththaatha da sound over a pooda kudaath...,1
4,ஆனவக் கொலைகள் நடக்காவிடில் ...நம் அடையாளம் அளி...,1
...,...,...
4227,Vijay going to next level thalaivaaaaa,0
4228,மண்ணு பொண்ணு ரெண்டுமே ஒன்னு அதுல எவன் கைய வச்ச...,1
4229,asuran= aadukalam+pudupettai+ wada chennai..ye...,0
4230,Vijay's all movies look like same.,0


# Classification Training (New labels)

In [4]:
# train_using_new_label
from simpletransformers.classification import ClassificationModel,ClassificationArgs
from simpletransformers.classification import MultiLabelClassificationModel

In [5]:
if EVALMODE==0:
    model_args = {
        "num_train_epochs":1,
        "overwrite_output_dir":True, 
        "manual_seed":42,
        "max_seq_length":150,
        #sliding_window=True,
        "train_batch_size":64,
        "eval_batch_size":64,
        "best_model_dir":"models/t1",
        "output_dir":"models/t1_max",
        "do_lower_case":True,
        "early_stopping_patience":2,
        "save_model_every_epoch":True,
        "early_stopping_consider_epochs":True,
        "use_early_stopping":False,
        "evaluate_during_training":True,
        #wandb_project='ST-Hope-CELoss',
        "learning_rate":3e-05}
    model = ClassificationModel(
        model_type='bert',
        model_name='bert-base-multilingual-uncased', 
        use_cuda=torch.cuda.is_available(),
        cuda_device="0",
        num_labels=2, 
        args=model_args)

    #     

In [8]:
if EVALMODE==0:
    model.train_model(df_test,args = model_args,eval_df=df_test)
elif EVALMODE==1:
    model_args = ClassificationArgs(
        eval_batch_size=512)
    model = ClassificationModel(
        model_type='bert',
        model_name='../models/baseline/offensive_baseline_mbert/', 
        use_cuda=torch.cuda.is_available(),
        cuda_device="0", args={"eval_batch_size":64})

In [ ]:
result, model_outputs, wrong_predictions = model.eval_model(df_test)
print(result)

  0%|          | 0/4232 [00:00<?, ?it/s]

In [9]:
testx = [i for i in df_test["text"].values.tolist()]
testx
predictions, raw_outputs = model.predict(testx)

ypr=[]
for i in predictions:
        ypr.append(i)

ygt=[]
for i in df_test["labels"].values:
        ygt.append(i)

print(classification_report(ygt,ypr,digits=5))
np.save("roberta_ypr_complex.npy",np.array(ypr))

  0%|          | 0/4232 [00:00<?, ?it/s]

  0%|          | 0/5 [00:00<?, ?it/s]

              precision    recall  f1-score   support

           0    0.87669   0.91599   0.89591      3190
           1    0.70189   0.60557   0.65018      1042

    accuracy                        0.83956      4232
   macro avg    0.78929   0.76078   0.77304      4232
weighted avg    0.83365   0.83956   0.83540      4232



In [9]:
# Interpret Predictions

In [11]:
from transformers import AutoModelForSequenceClassification, AutoTokenizer
from transformers_interpret import SequenceClassificationExplainer,MultiLabelClassificationExplainer
import pandas as pd
import os, torch
from sklearn.metrics import classification_report
import numpy as np
os.environ["KMP_DUPLICATE_LIB_OK"]="TRUE"

df_test = pd.read_csv("../data/span/test/test.tsv",sep="\t")
df_test



,text,spans
0,Evanukachum pondati kolandhainu sentiment irun...,"[71, 72, 73, 74, 75, 76, 77, 78, 79, 80, 81, 8..."
1,Adei ennada short film la irunthu suturukinga,"[34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45]"
2,Super dialogue Oruthar mela visvasam kattrath...,"[68, 69, 70, 71, 72, 73, 74, 75, 76, 77, 78, 7..."
3,Epadiya jathi padam. Ponnu padama edungada. In...,"[67, 68, 69, 70, 71, 72, 73, 74, 75, 76, 77, 7..."
4,Ponnu mella kaivaikuravan Kai mattum illa uyir...,"[42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 5..."
...,...,...
871,காலா.காபலி.அசுரன்.பாரியேரும்.பெருமள் வந்த அப்ப...,"[101, 102, 103, 104, 105, 106, 107, 108, 109, ..."
872,Kekka bekka short film mathri irukey,"[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11]"
873,Bayangaram... Trailerey ippadina appa Padam en...,"[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10]"
874,komali rasini vesam pottu tamil ilichavayangal...,"[0, 1, 2, 3, 4, 5, 6, 7, 31, 32, 33, 34, 35, 3..."


In [12]:
model_name = "models/baseline/offensive_baseline_mbert/"
model = AutoModelForSequenceClassification.from_pretrained(model_name)
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = model.to('cuda:0')
testx = [i for i in df_test["text"].values.tolist()]
cls_explainer = SequenceClassificationExplainer(
        model,
        tokenizer)

In [13]:
count=0
attribution_list_complex=[]

with torch.no_grad():
    for sntce in testx[:]:
           
        cls_explainer = SequenceClassificationExplainer(
        model,
        tokenizer)
        try:
            word_attributions = cls_explainer(sntce,internal_batch_size=1,n_steps=50)
        except:
            print("sentence unresolved -index",count)
            del cls_explainer
            torch.cuda.empty_cache()
            continue
        attribution_list_complex.append(word_attributions)
        count+=1
        if count%1 ==0:
            del cls_explainer
            torch.cuda.empty_cache()
            print("Completed", count, "sentences")
        word_attributions = None
    
np.save("bert_attribution_complex.npy",np.array(attribution_list_complex))

Completed 1 sentences
Completed 2 sentences
Completed 3 sentences
Completed 4 sentences
Completed 5 sentences
Completed 6 sentences
Completed 7 sentences
Completed 8 sentences
Completed 9 sentences
Completed 10 sentences
Completed 11 sentences
Completed 12 sentences
Completed 13 sentences
Completed 14 sentences
Completed 15 sentences
Completed 16 sentences
Completed 17 sentences
Completed 18 sentences
Completed 19 sentences
Completed 20 sentences
Completed 21 sentences
Completed 22 sentences
Completed 23 sentences
Completed 24 sentences
Completed 25 sentences
Completed 26 sentences
Completed 27 sentences
Completed 28 sentences
Completed 29 sentences
Completed 30 sentences
Completed 31 sentences
Completed 32 sentences
Completed 33 sentences
Completed 34 sentences
Completed 35 sentences
Completed 36 sentences
Completed 37 sentences
Completed 38 sentences
Completed 39 sentences
Completed 40 sentences
Completed 41 sentences
Completed 42 sentences
Completed 43 sentences
Completed 44 sentenc

Completed 347 sentences
Completed 348 sentences
Completed 349 sentences
Completed 350 sentences
Completed 351 sentences
Completed 352 sentences
Completed 353 sentences
Completed 354 sentences
Completed 355 sentences
Completed 356 sentences
Completed 357 sentences
Completed 358 sentences
Completed 359 sentences
Completed 360 sentences
Completed 361 sentences
Completed 362 sentences
Completed 363 sentences
Completed 364 sentences
Completed 365 sentences
Completed 366 sentences
Completed 367 sentences
Completed 368 sentences
Completed 369 sentences
Completed 370 sentences
Completed 371 sentences
Completed 372 sentences
Completed 373 sentences
Completed 374 sentences
Completed 375 sentences
Completed 376 sentences
Completed 377 sentences
Completed 378 sentences
Completed 379 sentences
Completed 380 sentences
Completed 381 sentences
Completed 382 sentences
Completed 383 sentences
Completed 384 sentences
Completed 385 sentences
Completed 386 sentences
Completed 387 sentences
Completed 388 se

Completed 689 sentences
Completed 690 sentences
Completed 691 sentences
Completed 692 sentences
Completed 693 sentences
Completed 694 sentences
Completed 695 sentences
Completed 696 sentences
Completed 697 sentences
Completed 698 sentences
Completed 699 sentences
Completed 700 sentences
Completed 701 sentences
Completed 702 sentences
Completed 703 sentences
Completed 704 sentences
Completed 705 sentences
Completed 706 sentences
Completed 707 sentences
Completed 708 sentences
Completed 709 sentences
Completed 710 sentences
Completed 711 sentences
Completed 712 sentences
Completed 713 sentences
Completed 714 sentences
Completed 715 sentences
Completed 716 sentences
Completed 717 sentences
Completed 718 sentences
Completed 719 sentences
Completed 720 sentences
Completed 721 sentences
Completed 722 sentences
Completed 723 sentences
Completed 724 sentences
Completed 725 sentences
Completed 726 sentences
Completed 727 sentences
Completed 728 sentences
Completed 729 sentences
Completed 730 se

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:25: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray


In [14]:
len(attribution_list_complex)

876

In [18]:
d = np.load("bert_attribution_complex.npy",allow_pickle=True)
d.shape

(876,)

# Lime interpreability

In [ ]:

# !python3 -m pip install lime

In [10]:
import numpy as np
from lime.lime_text import LimeTextExplainer
explainer = LimeTextExplainer(class_names=["0","1"])
df_test = pd.read_csv("../data/span/test/test.tsv",sep="\t")
testx = [i for i in df_test["text"].values.tolist()]

def softmax(x):
    e_x = np.exp(x - np.max(x))
    return e_x / e_x.sum(axis=0)

# define prediction function
def predict_probs(texts):
    predictions = model.predict(texts)
    x = np.array(list(predictions)[1])
    print(x.shape)
    return np.apply_along_axis(softmax, 1, x)

def explain(text):
    exp = explainer.explain_instance(text, predict_probs, num_features=6)
    return exp

In [ ]:
lime_attribution=[]
for txt in testx[:]:
    lime_attribution.append(explain(txt).as_list())
    
np.save("bert_lime_baseline.npy",np.array(lime_attribution))

/home/ubuntu/.local/lib/python3.6/site-packages/lime/lime_text.py:114: FutureWarning: split() requires a non-empty pattern match.
  self.as_list = [s for s in splitter.split(self.raw) if s]


  0%|          | 0/5000 [00:00<?, ?it/s]

  0%|          | 0/5 [00:00<?, ?it/s]

(5000, 2)


/home/ubuntu/.local/lib/python3.6/site-packages/lime/lime_text.py:114: FutureWarning: split() requires a non-empty pattern match.
  self.as_list = [s for s in splitter.split(self.raw) if s]


...

  0%|          | 0/5000 [00:00<?, ?it/s]

  0%|          | 0/5 [00:00<?, ?it/s]

(5000, 2)


/home/ubuntu/.local/lib/python3.6/site-packages/lime/lime_text.py:114: FutureWarning: split() requires a non-empty pattern match.
  self.as_list = [s for s in splitter.split(self.raw) if s]


  0%|          | 0/5000 [00:00<?, ?it/s]

In [ ]:
len(lime_attribution)

In [14]:
lime_attribution[0]

[('mavana', 0.06524792787778377),
 ('irundha', 0.044545596478418936),
 ('kollama', 0.04044303097989773),
 ('kolandhainu', 0.03240759471069187),
 ('pondati', 0.018613483552468228),
 ('poairu', 0.017821991347279603)]